# AgentCore Runtime での Runtime Context とセッション管理の理解

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime でランタイムコンテキストとセッション管理を理解し、操作する方法を学習します。この例では、AgentCore Runtime がセッションを処理し、複数の呼び出しにわたってコンテキストを維持し、エージェントがコンテキストオブジェクトを通じてランタイム情報にアクセスする方法を実証します。

Amazon Bedrock AgentCore Runtime は各ユーザーインタラクションに対して分離されたセッションを提供し、エージェントが複数の呼び出しにわたってコンテキストと状態を維持できる一方で、異なるユーザー間の完全なセキュリティ分離を確保します。

### チュートリアル詳細

|項目| 詳細|
|:--------------------|-|:---------------------------------------------------------------------------|
| チュートリアル タイプ | コンテキストとセッション管理|
| エージェント タイプ  | シングル         |
| エージェントフレームワーク | Strands Agents |
| LLM モデル          | Anthropic Claude Sonnet 3.7 |
| チュートリアル コンポーネント | Runtime Context、セッション管理、AgentCore Runtime、Strands Agent、Amazon Bedrock Model |
| チュートリアル領域   | 横断的                                                                   |
| サンプルの複雑さ      | 中級                                                                     |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK and boto3|

### チュートリアル アーキテクチャ

このチュートリアルでは、Amazon Bedrock AgentCore Runtime がセッションを管理し、エージェントにコンテキストを提供する方法を探索します。以下を実証します：

1. **セッション継続性**: 同じセッション ID が複数の呼び出しにわたってコンテキストを維持する方法
2. **コンテキストオブジェクト**: エージェントがコンテキストパラメーターを通じてランタイム情報にアクセスする方法
3. **セッション分離**: 異なるセッション ID が完全に分離された環境を作成する方法
4. **ペイロードの柔軟性**: ペイロードを通じてエージェントにカスタムデータを渡す方法

デモンストレーション目的として、これらのセッション管理機能を実証する Strands Agent を使用します。

    
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>

### チュートリアルの主な機能

* **セッションベースのコンテキスト管理**: AgentCore Runtime がセッション内でコンテキストを維持する方法の理解
* **Runtime セッションライフサイクル**: セッションの作成、維持、終了についての学習
* **コンテキストオブジェクトアクセス**: コンテキストパラメーターを通じたセッション ID などのランタイム情報へのアクセス
* **セッション分離**: 異なるセッションが完全な分離を提供する方法の実証
* **ペイロード処理**: カスタムペイロード構造を通じた柔軟なデータ渡し
* **呼び出し横断状態**: 同一セッション内での複数呼び出しにわたるエージェント状態の維持

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* 実行中の Docker

## Amazon Bedrock AgentCore Runtime セッションの理解

コードに入る前に、Amazon Bedrock AgentCore Runtime がセッションを管理する方法を理解することが重要です：

### セッション分離とセキュリティ

AgentCore Runtime は専用 microVM を通じて**完全なセッション分離**を提供します：

- **専用リソース**: 各セッションは分離された CPU、メモリ、ファイルシステムを持つ独自の microVM で実行
- **セキュリティ境界**: ユーザーセッション間の完全な分離により、データ汚染を防止
- **決定論的クリーンアップ**: セッション完了後、microVM は終了され、メモリがサニタイズされる

### セッションライフサイクル

AgentCore Runtime のセッションは特定のライフサイクルに従います：

1. **作成**: 一意の `runtimeSessionId` を持つ初回呼び出し時に新しいセッションを作成
2. **アクティブ状態**: セッションがリクエストを処理し、コンテキストを維持
3. **アイドル状態**: セッションがコンテキストを保持しながら次の呼び出しを待機
4. **終了**: 以下の理由でセッションが終了：
   - 非アクティブ（15分）
   - 最大寿命（8時間）
   - ヘルスチェック失敗

### コンテキスト持続性

セッション内で、AgentCore Runtime は以下を維持します：
- **会話履歴**: 過去のインタラクションとレスポンス
- **アプリケーション状態**: 実行中に作成された変数とオブジェクト
- **ファイルシステム**: セッション中に作成または変更されたファイル
- **環境変数**: カスタム設定と構成

### セッション管理のベストプラクティス

- **一意なセッション ID**: 各ユーザーまたは会話に対して一意のセッション ID を生成
- **コンテキスト再利用**: 関連する呼び出しに同じセッション ID を使用してコンテキストを維持
- **セッション境界**: 異なるユーザーまたは関連しない会話には異なるセッション ID を使用
- **一時的性質**: 永続的なデータストレージをセッションに依存しない（持続性には AgentCore Memory を使用）

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## AgentCore Runtime でのデプロイ用エージェントの準備

セッション管理とコンテキスト処理を実証するために、エージェントを AgentCore Runtime にデプロイしましょう。このエージェントは以下の方法を実演します：

1. **Runtime Context へのアクセス**: `context` パラメーターを使用してセッション情報を取得
2. **カスタムペイロードの処理**: ペイロードを通じて渡される構造化データの処理
3. **セッション状態の維持**: セッション内でのユーザーインタラクションの追跡
4. **セッション境界の実証**: 異なるセッションが分離されている方法の表示

### Context オブジェクトの理解

AgentCore Runtime の `context` オブジェクトは、現在の実行環境に関する重要な情報を提供します：

- **session_id**: 現在のランタイムセッション識別子
- **Runtime Metadata**: ランタイム環境に関する情報
- **実行詳細**: 現在の呼び出しに関するコンテキスト

### コンテキスト処理機能付き Strands Agent

セッション管理とコンテキスト処理を実証する実装を見てみましょう：

In [ ]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time.
    Always start by acknowledging the user's name 
    """
)

def get_user_name(user_id):
    users = {
        "1": "Maira",
        "2": "Mani",
        "3": "Mark",
        "4": "Ishan",
        "5": "Dhawal"
    }
    return users[user_id]
    
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    AgentCore Runtime entrypoint that demonstrates context handling and session management.
    
    Args:
        payload: The input payload containing user data and request information
        context: The runtime context object containing session and execution information
    
    Returns:
        str: The agent's response incorporating context information
    """
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # Access runtime context information
    print("=== Runtime Context Information ===")
    print("User id:", user_id)
    print("User Name:", user_name)
    print("User input:", user_input)
    print("Runtime Session ID:", context.session_id)
    print("Context Object Type:", type(context))
    print("=== End Context Information ===")
    
    # Create a personalized prompt that includes context information
    prompt = f"""My name is {user_name}. Here is my request: {user_input}
    
    Additional context: This is session {context.session_id}. 
    Please acknowledge my name and provide assistance."""
    
    response = agent(prompt)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

## AgentCore Runtime でのセッション管理の理解

上記のコードは、AgentCore Runtime がセッションを管理し、エージェントにコンテキストを提供する方法についていくつかの重要なコンセプトを実証します：

### Context オブジェクト構造

エントリーポイント関数の `context` パラメーターは、ランタイム情報へのアクセスを提供します：

```python
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    # セッション情報にアクセス
    session_id = context.session_id
    # エージェントロジックでコンテキスト情報を使用
```

### セッション継続性の利点

単一セッション内で、AgentCore Runtime は以下を提供します：

1. **永続的環境**: 変数と状態が呼び出しをまたいで持続
2. **コンテキスト保存**: エージェントが以前のインタラクションを参照可能
3. **リソース再利用**: 初期化されたモデルとツールが読み込まれたまま
4. **パフォーマンス利点**: 後続の呼び出しでのコールドスタート時間の短縮

### セッション分離保証

AgentCore Runtime はセッション間の完全な分離を確保します：

- **セキュリティ**: 各セッションは分離されたリソースを持つ独自の microVM で実行
- **プライバシー**: 異なるユーザーセッション間でのデータ漏洩なし
- **信頼性**: 1つのセッションの問題が他に影響しない
- **クリーンアップ**: セッション終了後の完全なメモリサニタイゼーション

### ペイロードの柔軟性

`payload` パラメーターは柔軟なデータ受け渡しを可能にします：

```python
# ペイロード構造の例
payload = {
    "prompt": "ユーザーの質問",
    "user_id": "1",
    "preferences": {...},
    "context_data": {...}
}
```

これにより、ランタイムが提供するセッションコンテキストを維持しながら、クライアントとエージェント間の豊富で構造化された通信が可能になります。

### AgentCore Runtime デプロイメントの設定

次に、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップ中に、アプリケーションコードに基づいて docker ファイルが生成されます

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_context"
)

### コンテキスト対応エージェントの AgentCore Runtime への起動

docker ファイルができたので、コンテキスト対応エージェントを AgentCore Runtime に起動しましょう。これにより Amazon ECR リポジトリと AgentCore Runtime が作成されます。

このエージェントは、AgentCore Runtime がセッションを管理し、エージェントにコンテキスト情報を提供する方法を実証します。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

### AgentCore Runtime ステータスの確認
AgentCore Runtime をデプロイしたので、そのデプロイメントステータスを確認しましょう

In [ ]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## セッション管理とコンテキスト処理の実証

次に、さまざまなシナリオをテストして AgentCore Runtime の主要なセッション管理機能を実証しましょう：

### シナリオ 1: セッション継続性
同じセッション ID を複数の呼び出しで使用して、コンテキストがどのように維持されるかを示します。

### シナリオ 2: セッション分離
異なるセッション ID を使用して、セッション間の完全な分離を実証します。

### シナリオ 3: コンテキスト情報アクセス
エージェントがランタイムコンテキスト情報にどのようにアクセスできるかを示します。

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [ ]:
import uuid
import json
from IPython.display import Markdown, display

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()
print(f"📋 Starting Session 1: {session_id}")
print(f"👤 User: Maira (ID: 1)")
print(f"❓ First question about weather\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather outside?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# Continue with the same session ID to demonstrate session continuity
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Follow-up question about math\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How much is 2X5?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# Continue with the same session ID - notice how the agent remembers the previous calculation
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Building on previous answer - demonstrates context continuity\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "and that plus 34?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# NEW SESSION - Demonstrate session isolation
# Create a completely new session ID to show that context is lost
new_session_id = uuid.uuid4()
print(f"🆕 Starting NEW Session 2: {new_session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Attempting to reference previous calculation - should fail due to session isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "And plus 10?",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

In [ ]:
# NEW SESSION AND USER - Demonstrate complete isolation
different_user_session = uuid.uuid4()
print(f"🆕 Starting Session 3: {different_user_session}")
print(f"👤 Different user: Mani (ID: 2)")
print(f"❓ Same question as first user - demonstrates user isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather?",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = json.loads(invoke_response['response'][0])
display(Markdown(response_data))

## セッション管理結果の理解

上記の実証は、AgentCore Runtime のセッション管理のいくつかの重要な側面を実演しています：

### 1. セッション継続性（セッション 1）
- **最初の呼び出し**: エージェントが天気の質問に回答し、ユーザー名を確認
- **2回目の呼び出し**: エージェントが計算を実行（2×5=10）
- **3回目の呼び出し**: エージェントが以前の結果を参照（「それに34を足すと」= 44）

**重要な学習**: エージェントは同じセッション内の複数の呼び出しにわたってコンテキストを維持し、以前のインタラクションの計算結果を記憶していました。

### 2. セッション分離（セッション 2）
- **新しいセッション ID**: 完全に新しいセッションを作成
- **同じユーザー**: 同じユーザー ID だが異なるセッションを使用
- **コンテキストの損失**: エージェントは以前の計算を参照できない

**重要な学習**: 同じユーザーでも、新しいセッションは以前のコンテキストにアクセスできない完全に分離された環境を作成します。

### 3. ユーザーとセッションの分離（セッション 3）
- **異なるユーザー**: Maira の代わりに Mani
- **新しいセッション**: 以前のセッションからの完全な分離
- **新鮮なコンテキスト**: エージェントはクリーンな状態で開始

**重要な学習**: 各セッションは完全な分離を提供し、異なるユーザーとインタラクション間のプライバシーとセキュリティを確保します。

### 4. Context オブジェクトの使用
すべての呼び出しを通じて、エージェントは：
- `context.session_id` を通じてランタイムコンテキストにアクセス
- カスタムペイロードデータ（`user_id`、`prompt`）を処理
- ログと デバッグ情報を維持

**重要な学習**: コンテキストオブジェクトは、エージェントが機能強化とデバッグのために使用できる貴重なランタイム情報を提供します。

### 実証されたセッション管理のベストプラクティス

1. **会話の継続性のために一貫したセッション ID を使用**
2. **異なるユーザーまたは会話には一意のセッション ID を生成**
3. **強化されたエージェント動作のためにコンテキスト情報を活用**
4. **セッション境界を考慮した設計** - セッション間の持続性を前提としない
5. **セッション変更や期限切れ時の適切なコンテキスト損失を処理**

## クリーンアップ（オプション）

作成された AgentCore Runtime をクリーンアップしましょう

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# おめでとうございます！

Amazon Bedrock AgentCore Runtime を使用したセッション管理とコンテキスト処理の実装とテストに成功しました！

## 学んだこと：

### セッション管理の基礎
* **セッション継続性**: 同じセッション ID が複数の呼び出しにわたってコンテキストを維持する方法
* **セッション分離**: 異なるセッション ID が完全に分離された環境を作成する方法
* **コンテキスト保存**: エージェントが状態を維持し、以前のインタラクションを参照できる方法
* **セキュリティ境界**: AgentCore Runtime がユーザー間の完全な分離を確保する方法

### Runtime Context の処理
* **Context オブジェクトアクセス**: `context` パラメーターを通じたランタイム情報へのアクセス方法
* **セッション情報**: エージェントロジックでセッション ID を取得・使用する方法
* **ペイロード処理**: カスタムペイロードを通じて渡される構造化データの処理方法
* **Runtime Metadata**: エージェントが実行環境情報にアクセスする方法

### AgentCore Runtime アーキテクチャ
* **MicroVM 分離**: 各セッションが独自の分離された microVM で実行される仕組み
* **リソース管理**: セッションごとの専用 CPU、メモリ、ファイルシステム
* **セキュリティモデル**: セッション終了後の完全なメモリサニタイゼーション
* **ライフサイクル管理**: セッション状態（アクティブ、アイドル、終了）とタイムアウト

### ベストプラクティスの実装
* **セッション ID 生成**: 異なる会話に対する一意識別子の作成
* **コンテキスト活用**: 強化されたエージェント動作のためのランタイムコンテキストの利用
* **状態管理**: 一時的対持続的状態の理解
* **エラー処理**: コンテキスト損失とセッション境界の適切な処理